In [114]:
import os 
import pandas as pd
from pathlib import Path

from survey_util import load_raw_survey_data, count_by_school_type, calculate_percentage, drop_duplicates

In [115]:
# Define paths and base directories

DATA_DIR = Path('../../data')
SURVEY_DIR = DATA_DIR / 'survey'
RAW_DIR = SURVEY_DIR / 'raw'

VIZ_DIR = Path('../../src/_data/viz/survey/phase_2')

raw_data = load_raw_survey_data()

# Handle duplicates
raw_data_deduped = drop_duplicates(raw_data)

base_respondents = len(raw_data)

SCHOOLS_DATA = pd.read_csv(Path(DATA_DIR, 'leeds_schools_public.csv'))

dfe_data = pd.read_csv(Path(DATA_DIR, 'spc_ees_school_characteristics.csv'), usecols={
    'time_period', 
    'la_name',
    'school_urn',
    'school_name',
    'phase_type_grouping',
    'typeofestablishment_name',
    'ward_code',
    'ward_name',
    'school_postcode',
    'school_size'
})


In [116]:
headline_questions = [
    '05_overall_rating_arts_offer', #excellent good
    '06_pupil_arts_entitlement', # yes no 
    '07_additional_arts_funding', #yesno
    '10_signposting_beyond_curriculum', #yesno
    # '13_arts_facilities', #list of facilities
    '14_rating_extra_curricular_arts_enrichment_offering', #excellent good
    '16_arts_award', #yesno
    '18_external_orgs_arts_performances', #yesno
    # '18a_arts_performance_external_org_subject_areas', #list of subjects
    '24_partnerships_external_orgs', #yesno
]


Headline stats

In [117]:

# Calculate values for headlines, out of total survey responses
excellent_arts_curriculum = int(((raw_data['05_overall_rating_arts_offer'].value_counts().Excellent) / base_respondents * 100))
pupil_arts_entitlement = int((len(raw_data.loc[raw_data['06_pupil_arts_entitlement']=='Yes']) / base_respondents * 100))
additional_arts_funding = int((len(raw_data.loc[raw_data['07_additional_arts_funding']=='Yes']) / base_respondents * 100))
signposting_beyond_curriculum = int((len(raw_data.loc[raw_data['10_signposting_beyond_curriculum']=='Yes']) / base_respondents * 100))
excellent_extra_curricular_arts = int(((raw_data['14_rating_extra_curricular_arts_enrichment_offering'].value_counts().Excellent) / base_respondents * 100))
arts_award = int((len(raw_data.loc[raw_data['16_arts_award']=='Yes']) / base_respondents * 100))
external_orgs_arts_performances = int((len(raw_data.loc[raw_data['18_external_orgs_arts_performances']=='Yes']) / base_respondents * 100))
partnerships_external_orgs = int((len(raw_data.loc[raw_data['24_partnerships_external_orgs']=='Yes']) / base_respondents * 100))

headlines_data = {
    'Title': [
        "Arts curriculum",
        "Arts entitlement",
        "Additional arts funding",
        "Signposting",
        "Extra-curricular offering",
        "Arts award",
        "External practitioners",
        "Local partnerships",
    ],
    'Value': [
        excellent_arts_curriculum,
        pupil_arts_entitlement,
        additional_arts_funding,
        signposting_beyond_curriculum,
        excellent_extra_curricular_arts,
        arts_award,
        external_orgs_arts_performances,
        partnerships_external_orgs
    ],
    'Note': [
        "Rated their arts curriculum as Excellent",
        "Said they have a commitment to pupil entitlement in the arts",
        "Said they seek additional funding for arts curriculum and enrichment",
        "Said they sign-post additional creative opportunities for students",
        "Rated their extra-curricular arts offering as Excellent",
        "Said their school offers an Arts Award",
        "Said they invited external creative practitioners into the school",
        "Said their school has a partnership with local arts organisations"
    ],
    'Suffix': '%'
}

headlines_index = headline_questions[:8]

headlines = pd.DataFrame(headlines_data, index=pd.Index(headlines_index, name='Question'))

headlines.to_csv(os.path.join(VIZ_DIR, 'headlines.csv'))


## Overview of Respondents

Survey responses by school type

In [118]:
# Responses broken down by school type

by_school_type = pd.DataFrame({
    'total_respondents' : raw_data_deduped.groupby('03_school_type')['03_school_type'].count(),
    'percent_respondents': raw_data_deduped.groupby('03_school_type')['03_school_type'].count().div(base_respondents).mul(100).round(0).astype(int),
    'unit': '%',
    'notes': [
        'of survey respondents were from a Primary school',
        'of survey respondents were from a Secondary school',
        'of survey respondents were from a Special school',
        'of survey respondents were from a Through school'
    ]
}).to_csv(os.path.join(VIZ_DIR, 'by_school_type.csv'))

## Arts Provision in Schools

Commitment to the arts

In [119]:
# 06_pupil_arts_entitlement

pupil_arts_entitlement = (
    raw_data_deduped.groupby(['03_school_type', '06_pupil_arts_entitlement'])
    .size()
    .unstack(fill_value=0)
    .apply(lambda x: (x / x.sum()) * 100, axis=1)
    .round(0)
    .astype(int)
    .reset_index()
    .rename(columns={'03_school_type': 'School Type'})
)

pupil_arts_entitlement['Suffix'] = '%'
pupil_arts_entitlement.to_csv(Path(VIZ_DIR, 'pupil_arts_entitlement.csv'), index=False)


In [120]:
# 05_overall_rating_arts_offer - deduped data
overall_rating_arts_offer = (
    raw_data_deduped.groupby(['03_school_type', '05_overall_rating_arts_offer'])
    .size()
    .unstack(fill_value=0)
    .apply(lambda x: (x / x.sum()) * 100, axis=1)
    .round(0)
    .astype(int)
    .reset_index()
    .rename(columns={'03_school_type': 'School Type'})
    .pivot_table(columns='School Type')
    .reset_index()
    .rename(columns={'index': 'Rating'})
)

overall_rating_arts_offer['Suffix'] = '%'
overall_rating_arts_offer.to_csv(Path(VIZ_DIR, 'overall_rating_arts_offer.csv'), index=False)


In [121]:
# 14_rating_extra_curricular_arts_enrichment_offering - deduped data

rating_extra_curricular_arts_enrichment_offering = (
    raw_data_deduped.groupby(['03_school_type', '14_rating_extra_curricular_arts_enrichment_offering'])
    .size()
    .unstack(fill_value=0)
    .apply(lambda x: (x / x.sum()) * 100, axis=1)
    .round(0)
    .astype(int)
    .reset_index()
    .rename(columns={'03_school_type': 'School Type'})
    .pivot_table(columns='School Type')
    .reset_index()
    .rename(columns={'index': 'Rating'})
)

rating_extra_curricular_arts_enrichment_offering['Suffix'] = '%'
rating_extra_curricular_arts_enrichment_offering.to_csv(Path(VIZ_DIR, 'rating_extra_curricular_arts_enrichment_offering.csv'), index=False)


Signposting headlines

In [122]:
# Signposting headlines - percentage of total survey responses

signposting = raw_data.loc[raw_data['10_signposting_beyond_curriculum']=='Yes']
competitions = raw_data.loc[raw_data['09_competitions_showcases']=='Yes']

responses = pd.DataFrame({
    'Competitions & Showcases' : [int(len(competitions)/base_respondents*100)],
    'Signposting beyond the curriculum': [int(len(signposting)/base_respondents*100)]
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'of all respondents said their school actively sign-posts for pupils to develop creative skills and participate in arts opportunities beyond the curriculum',
        'of all respondents said their school enters pupils into competitions or showcases - whether that is at a local community or regional/national level'
    ]

signposting_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'signposting_headlines.csv'), index=False)


### Opinions on the arts curriculum

Do you consider your school to lack any particular areas of expertise in delivering the arts curriculum?

Do you consider your school to be an exemplar in any areas of the arts curriculum?

In [123]:
# 34_school_exemplar_arts_curriculum
# 33_school_lacking_expertise_arts_curriculum

school_exemplar_arts_curriculum = raw_data.loc[raw_data['34_school_exemplar_arts_curriculum']=='Yes']
school_lacking_expertise_arts_curriculum = raw_data.loc[raw_data['33_school_lacking_expertise_arts_curriculum']=='Yes']

responses = pd.DataFrame({
    'Curriculum Exemplar' : [int(len(school_exemplar_arts_curriculum)/base_respondents*100)],
    'Lacking Expertise': [int(len(school_lacking_expertise_arts_curriculum)/base_respondents*100)],
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'of all respondents said they consider their school to be an exemplar in areas of the arts curriculum',
        'of all respondents said they consider their school to lack areas of expertise in delivering the arts curriculum',
    ]

signposting_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'school_expertise_headlines.csv'), index=False)


print(school_exemplar_arts_curriculum['34_school_exemplar_arts_curriculum'])

unique_response_number
1132118-1132100-122818827    Yes
Name: 34_school_exemplar_arts_curriculum, dtype: object


External practitioners

In [124]:
partnerships_external_orgs = raw_data.loc[raw_data['24_partnerships_external_orgs']=='Yes']
external_orgs_arts_performances = raw_data.loc[raw_data['18_external_orgs_arts_performances']=='Yes']
partnerships_local_schools = raw_data.loc[raw_data['25_partnerships_local_schools']=='Yes']

responses = pd.DataFrame({
    'Partnerships' : [int(len(partnerships_external_orgs)/base_respondents*100)],
    'Delivering activities': [int(len(external_orgs_arts_performances)/base_respondents*100)],
    'School partnerships': [int(len(partnerships_local_schools)/base_respondents*100)]
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'of all respondents said their school has a partnership with any local arts organisations',
        'of all respondents said their school invited external practitioners to deliver creative activities',
        'of all respondents said their school partners with other schools to support the arts curriculum'
    ]

signposting_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'external_partnerships_headlines.csv'), index=False)

Subject areas of the organisations and creative practitioners engaging with schools:

In [125]:
# Subject areas of external practitioners - deduped data

subject_areas = raw_data_deduped[['02_school_name', '03_school_type', '18a_arts_performance_external_org_subject_areas']].dropna().reset_index()

subject_areas['18a_arts_performance_external_org_subject_areas'] = subject_areas['18a_arts_performance_external_org_subject_areas'].str.split(',')
subject_areas = subject_areas.explode('18a_arts_performance_external_org_subject_areas').drop(columns={'unique_response_number'})

subject_areas = subject_areas.groupby(['03_school_type', '18a_arts_performance_external_org_subject_areas']).count().reset_index()

subject_areas = subject_areas.rename(columns={
    '03_school_type': 'School Type',
    '18a_arts_performance_external_org_subject_areas': 'Subject Area',
    '02_school_name': 'Count'
})

counts_by_school_type = count_by_school_type(raw_data_deduped)

subject_areas['Percent'] = subject_areas.apply(lambda row: calculate_percentage(row, counts_by_school_type), axis=1)

subject_areas = subject_areas.pivot_table(columns=['School Type'], index=['Subject Area'], values=['Percent']).reset_index().set_index('Subject Area')

subject_areas.columns = ['Primary', 'Secondary', 'Special School', 'Through School']

subject_areas = subject_areas.fillna(0).astype(int)

subject_areas.to_csv(Path(VIZ_DIR, 'subject_areas_headlines.csv'), index=True)

subject_areas


,Primary,Secondary,Special School,Through School
Subject Area,,,,
Art,29,33,33,50
Dance,31,67,17,50
Design and Technology,14,17,0,0
Digital media and gaming,3,33,0,50
Film,3,0,0,0
Museum/Heritage,17,50,17,50
Music,51,67,50,100
Theatre,40,67,83,50


Community arts programmes:

In [126]:
# Community programs - total survey responses

community_based_arts_programmes = raw_data.loc[raw_data['26_community_based_arts_programmes']=='Yes']
national_arts_programmes = raw_data.loc[raw_data['23_national_programmes']=='Yes']

responses = pd.DataFrame({
    'Community-based arts programmes' : [int(len(community_based_arts_programmes)/base_respondents*100)],
    'National arts programmes' : [int(len(national_arts_programmes)/base_respondents*100)],
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'of all respondents said their school is part of a local/community-based arts programme.',
        'of all respondents said their school is part of a national arts programme.',
    ]

signposting_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'community_programs_headlines.csv'), index=False)

## Facilities

In [127]:
# Facilities - deduped data

facilities = raw_data_deduped[['02_school_name', '03_school_type', '13_arts_facilities']].dropna().reset_index()


facilities['13_arts_facilities'] = facilities['13_arts_facilities'].str.split(',')
facilities = facilities.explode('13_arts_facilities').drop(columns={'unique_response_number'})

facilities = facilities.groupby(['03_school_type', '13_arts_facilities']).size().reset_index(name='Count')

counts_by_school_type = count_by_school_type(raw_data_deduped)

facilities['Total'] = facilities.groupby('03_school_type')['Count'].transform('sum').round(0)

facilities['Percent'] = ((facilities['Count'] / facilities['Total']) * 100).round(0).astype(int)

facilities_pivot = facilities.pivot_table(index='13_arts_facilities', columns='03_school_type', values='Percent', fill_value=0)

# facilities_pivot['Total'] = facilities_pivot.sum(axis=1).round(0).astype(int)

facilities_pivot.reset_index(inplace=True)

facilities_pivot.columns.name = None 
facilities_pivot.rename(columns={'Primary': 'Primary', 'Secondary': 'Secondary', 'Special': 'Special School', 'Through': 'Through School'}, inplace=True)

facilities_pivot.to_csv(Path(VIZ_DIR, 'facilities.csv'), index=False)

facilities


,03_school_type,13_arts_facilities,Count,Total,Percent
0,Primary,A music studio,1,46,2
1,Primary,A theatre/hall space with production facilities,14,46,30
2,Primary,Art studios,3,46,7
3,Primary,Music lesson/practice rooms,13,46,28
4,Primary,No arts/creative facilities,12,46,26
5,Primary,Other arts/creative facilities,3,46,7
6,Secondary,A drama studio,6,31,19
7,Secondary,A music studio,5,31,16
8,Secondary,A theatre/hall space with production facilities,6,31,19
9,Secondary,Art studios,6,31,19


## Funding & Support

Does your school attract addition funding for arts curriculum or enrichment activity?

In [128]:
# 07_additional_arts_funding

funding = raw_data_deduped[['02_school_name', '03_school_type', '07_additional_arts_funding']].dropna().reset_index()

primary_schools = funding.loc[(funding['07_additional_arts_funding'] =='Yes') & (funding['03_school_type'] =='Primary')]
secondary_schools = funding.loc[(funding['07_additional_arts_funding'] =='Yes') & (funding['03_school_type'] =='Secondary')]
through_schools = funding.loc[(funding['07_additional_arts_funding'] =='Yes') & (funding['03_school_type'] =='Through School')]
special_schools = funding.loc[(funding['07_additional_arts_funding'] =='Yes') & (funding['03_school_type'] =='Special School')]

counts_by_school_type = count_by_school_type(raw_data_deduped)

responses = pd.DataFrame({
    'Primary' : [int(len(primary_schools)/(counts_by_school_type[counts_by_school_type['03_school_type']=='Primary'].Count.item())*100)],
    'Secondary' : [int(len(secondary_schools)/(counts_by_school_type[counts_by_school_type['03_school_type']=='Secondary'].Count.item())*100)],
    'Through Schools' : [int(len(through_schools)/(counts_by_school_type[counts_by_school_type['03_school_type']=='Through School'].Count.item())*100)],
    'Special Schools' : [int(len(special_schools)/(counts_by_school_type[counts_by_school_type['03_school_type']=='Special School'].Count.item())*100)],
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'of all Primary schools said their schools attracts additional funding for arts curriculum or enrichment activity.',
        'of all Secondary schools said their schools attracts additional funding for arts curriculum or enrichment activity.',
        'of all Through schools said their schools attracts additional funding for arts curriculum or enrichment activity.',
        'of all Special schools said their schools attracts additional funding for arts curriculum or enrichment activity.',
    ]

funding_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'funding_headlines.csv'), index=False)

Funding sources for provision of art curriculum and enrichment activity:

In [129]:
# 07a_arts_funding_sources

funding_sources = raw_data_deduped[['02_school_name', '03_school_type', '07a_arts_funding_sources']].dropna().reset_index()

funding_sources['07a_arts_funding_sources'] = funding_sources['07a_arts_funding_sources'].str.split(',')
funding_sources = funding_sources.explode('07a_arts_funding_sources').drop(columns={'unique_response_number'})

funding_sources = funding_sources.groupby(['03_school_type', '07a_arts_funding_sources']).count().reset_index()

funding_sources = funding_sources.rename(columns={
    '03_school_type': 'School Type',
    '07a_arts_funding_sources': 'funding_sources',
    '02_school_name': 'Count'
})

counts_by_school_type = count_by_school_type(raw_data_deduped)

funding_sources['Percent'] = funding_sources.apply(lambda row: calculate_percentage(row, counts_by_school_type), axis=1)

funding_sources_pivot = funding_sources.pivot_table(columns=['School Type'], index=['funding_sources'], values=['Percent']).reset_index().set_index('funding_sources')

funding_sources_pivot.columns = ['Primary', 'Secondary', 'Special School', 'Through School']

funding_sources_pivot = funding_sources_pivot.fillna(0).astype(int)

funding_sources_pivot['Suffix'] = '%'

funding_sources_pivot.to_csv(Path(VIZ_DIR, 'funding_sources_headlines.csv'), index=True)

funding_sources_pivot


,Primary,Secondary,Special School,Through School,Suffix
funding_sources,,,,,
City Council,6,17,17,0,%
Fundraising events or activities,3,17,17,0,%
Other,0,17,17,50,%
Parents,6,17,0,0,%
Trusts and foundations,3,17,33,50,%


## Trips

Does your school take pupils on arts trips outside the school?

In [130]:
# 19_arts_trips

trips = raw_data_deduped[['02_school_name', '03_school_type', '19_arts_trips']].dropna().reset_index()

primary_schools = trips.loc[(trips['19_arts_trips'] =='Yes') & (trips['03_school_type'] =='Primary')]
secondary_schools = trips.loc[(trips['19_arts_trips'] =='Yes') & (trips['03_school_type'] =='Secondary')]
through_schools = trips.loc[(trips['19_arts_trips'] =='Yes') & (trips['03_school_type'] =='Through School')]
special_schools = trips.loc[(trips['19_arts_trips'] =='Yes') & (trips['03_school_type'] =='Special School')]

counts_by_school_type = count_by_school_type(raw_data_deduped)

responses = pd.DataFrame({
    'Primary' : [int(len(primary_schools)/(counts_by_school_type[counts_by_school_type['03_school_type']=='Primary'].Count.item())*100)],
    'Secondary' : [int(len(secondary_schools)/(counts_by_school_type[counts_by_school_type['03_school_type']=='Secondary'].Count.item())*100)],
    'Through Schools' : [int(len(through_schools)/(counts_by_school_type[counts_by_school_type['03_school_type']=='Through School'].Count.item())*100)],
    'Special Schools' : [int(len(special_schools)/(counts_by_school_type[counts_by_school_type['03_school_type']=='Special School'].Count.item())*100)],
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'of all Primary schools said their school takes pupils on arts trips outside the school.',
        'of all Secondary schools said their school takes pupils on arts trips outside the school.',
        'of all Through schools said their school takes pupils on arts trips outside the school.',
        'of all Special schools said their school takes pupils on arts trips outside the school.',
    ]

arts_trips_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'arts_trips_headlines.csv'), index=False)


Typically, how are your school's arts trips funded?

In [131]:
# 19a_arts_trips_funding

arts_trips_funding = raw_data_deduped[['02_school_name', '03_school_type', '19a_arts_trips_funding']].dropna().reset_index()

arts_trips_funding['19a_arts_trips_funding'] = arts_trips_funding['19a_arts_trips_funding'].str.split(',')
arts_trips_funding = arts_trips_funding.explode('19a_arts_trips_funding').drop(columns={'unique_response_number'})

arts_trips_funding = arts_trips_funding.groupby(['03_school_type', '19a_arts_trips_funding']).count().reset_index()

arts_trips_funding = arts_trips_funding.rename(columns={
    '03_school_type': 'School Type',
    '19a_arts_trips_funding': 'arts_trips_funding',
    '02_school_name': 'Count'
})

counts_by_school_type = count_by_school_type(raw_data_deduped)

arts_trips_funding['Percent'] = arts_trips_funding.apply(lambda row: calculate_percentage(row, counts_by_school_type), axis=1)

arts_trips_funding['arts_trips_funding'] = arts_trips_funding['arts_trips_funding'].str.replace('â€™', "'")
arts_trips_funding['arts_trips_funding'] = arts_trips_funding['arts_trips_funding'].str.replace('From core school funding', "Core school funding")

arts_trips_funding_pivot = arts_trips_funding.pivot_table(columns=['School Type'], index=['arts_trips_funding'], values=['Percent']).reset_index().set_index('arts_trips_funding')

arts_trips_funding_pivot.columns = ['Primary', 'Secondary', 'Special School', 'Through School']

arts_trips_funding_pivot = arts_trips_funding_pivot.fillna(0).astype(int)

arts_trips_funding_pivot['Suffix'] = '%'

arts_trips_funding_pivot.to_csv(Path(VIZ_DIR, 'arts_trips_funding_headlines.csv'), index=True)

arts_trips_funding_pivot

,Primary,Secondary,Special School,Through School,Suffix
arts_trips_funding,,,,,
Additional funding,11,33,33,0,%
Core school funding,49,83,50,50,%
Don't know,6,0,0,0,%
Parent funding,54,50,17,50,%


Which of the following did you undertake in academic year 2022/23?

In [132]:
# 20_trips_undertaken_22_23

trips_undertaken = raw_data_deduped[['02_school_name', '03_school_type', '20_trips_undertaken_22_23']].dropna().reset_index()

trips_undertaken['20_trips_undertaken_22_23'] = trips_undertaken['20_trips_undertaken_22_23'].str.split(',')
trip_types = trips_undertaken.explode('20_trips_undertaken_22_23').drop(columns={'unique_response_number'})

trip_types = trip_types.groupby(['03_school_type', '20_trips_undertaken_22_23']).count().reset_index()

trip_types = trip_types.rename(columns={
    '03_school_type': 'School Type',
    '20_trips_undertaken_22_23': 'trip_types',
    '02_school_name': 'Count'
})

counts_by_school_type = count_by_school_type(raw_data_deduped)

trip_types['Percent'] = trip_types.apply(lambda row: calculate_percentage(row, counts_by_school_type), axis=1)

trip_types['trip_types'] = trip_types['trip_types'].str.replace("(", '').str.replace(")", '').str.replace(
    "Trips to a library other than your own school library", 'Trips to a library')

trip_types_pivot = trip_types.pivot_table(columns=['School Type'], index=['trip_types'], values=['Percent']).reset_index().set_index('trip_types')

trip_types_pivot.columns = ['Primary', 'Secondary', 'Special School', 'Through School']

trip_types_pivot = trip_types_pivot.fillna(0).astype(int)

trip_types_pivot['Suffix'] = '%'

trip_types_pivot.to_csv(Path(VIZ_DIR, 'trip_types_headlines.csv'), index=True)


C:\Users\oi-user\AppData\Local\Temp\ipykernel_9804\2186878262.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  trip_types['trip_types'] = trip_types['trip_types'].str.replace("(", '').str.replace(")", '').str.replace(


Threatre trip locations...

In [133]:
# 20b_theatre_trip_locations

theatre_trips = raw_data_deduped[['02_school_name', '03_school_type', '20b_theatre_trip_locations']].dropna().reset_index()

theatre_trips['20b_theatre_trip_locations'] = theatre_trips['20b_theatre_trip_locations'].str.split(',')
theatre_trip_locations = theatre_trips.explode('20b_theatre_trip_locations').drop(columns={'unique_response_number'})

theatre_trip_locations = theatre_trip_locations.groupby(['03_school_type', '20b_theatre_trip_locations']).count().reset_index()

theatre_trip_locations = theatre_trip_locations.rename(columns={
    '03_school_type': 'School Type',
    '20b_theatre_trip_locations': 'theatre_trip_locations',
    '02_school_name': 'Count'
})

counts_by_school_type = count_by_school_type(raw_data_deduped)

theatre_trip_locations['Percent'] = theatre_trip_locations.apply(lambda row: calculate_percentage(row, counts_by_school_type), axis=1)

theatre_trip_locations_pivot = theatre_trip_locations.pivot_table(columns=['School Type'], index=['theatre_trip_locations'], values=['Percent']).reset_index().set_index('theatre_trip_locations')

theatre_trip_locations_pivot.columns = ['Primary', 'Secondary', 'Special School', 'Through School']

theatre_trip_locations_pivot = theatre_trip_locations_pivot.fillna(0).astype(int)

theatre_trip_locations_pivot['Suffix'] = '%'

theatre_trip_locations_pivot.to_csv(Path(VIZ_DIR, 'theatre_trip_locations_headlines.csv'), index=True)

## Barriers

Regarding access, diversity, and inclusion, does your school experience any barriers or challenges in encouraging, recruiting, or gaining permissions for pupils to go on trips out of school?

Regarding access, diversity, and inclusion, does your school experience any barriers or challenges with the experience or facilities at host venues when taking pupils on trips out of the school?

In [137]:
# 21_barriers_encouraging_recruiting_permissions_trips
# 22_barriers_facilities_host_venues_trips

barriers_encouraging_recruiting_permissions_trips = raw_data.loc[raw_data['21_barriers_encouraging_recruiting_permissions_trips']=='Yes']
barriers_facilities_host_venues_trips = raw_data.loc[raw_data['22_barriers_facilities_host_venues_trips']=='Yes']

responses = pd.DataFrame({
    'Barriers with trips' : [int(len(barriers_encouraging_recruiting_permissions_trips)/base_respondents*100)],
    'Barriers with host venues': [int(len(barriers_facilities_host_venues_trips)/base_respondents*100)],
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'of all respondents said their school has experienced barriers in facilitating trips out of school',
        'of all respondents said their school has experienced barriers or challenges with the experience at host venues',
    ]

signposting_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'barriers_headlines.csv'), index=False)

In [140]:
# 28_partnerships_diversity_challenges

partnerships_diversity_challenges = raw_data.loc[raw_data['28_partnerships_diversity_challenges']=='Yes']

responses = pd.DataFrame({
    'Diversity challenges' : [int(len(partnerships_diversity_challenges)/base_respondents*100)],
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'of all respondents said their school is able to find arts delivery partners who reflect the diversity of their pupils',
    ]

signposting_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'diversity_challenges_headline.csv'), index=False)



## Careers

Careers & student progression headlines

In [143]:
# 30_specialist_creative_career_advice
# 29_pupil_practitioner_engagement_opportunities
# 32_supporting_teacher_cpd
# 31_data_pupil_progression_creative_education

specialist_creative_career_advice = raw_data.loc[raw_data['30_specialist_creative_career_advice']=='Yes']
pupil_practitioner_engagement_opportunities = raw_data.loc[raw_data['29_pupil_practitioner_engagement_opportunities']=='Yes']
supporting_teacher_cpd = raw_data.loc[raw_data['32_supporting_teacher_cpd']=='Yes']
data_pupil_progression_creative_education = raw_data.loc[raw_data['31_data_pupil_progression_creative_education']=='Yes']


responses = pd.DataFrame({
    'Creative career advice' : [int(len(specialist_creative_career_advice)/base_respondents*100)],
    'Practitioner engagement opportunities': [int(len(pupil_practitioner_engagement_opportunities)/base_respondents*100)],
    'Supporting teacher CPD': [int(len(supporting_teacher_cpd)/base_respondents*100)],
    'Pupil Progression' : [int(len(data_pupil_progression_creative_education)/base_respondents*100)],
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'of all respondents said their school have someone able to give specialist advice on career pathways into the arts',
        'of all respondents said their school gives pupils opportunities to engage with arts practitioners',
        "of all respondents said their school support teachers' CPD in arts subjects",
        'of all respondents said their school holds data on pupil progression to study arts in further or higher education',
    ]

signposting_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'careers_headlines.csv'), index=False)

